In [242]:
import openai

openai.api_base = "https://free.v36.cm/v1"
openai.api_key = "sk-RFCn5XAIanQqIw5eB0C58d15328246769b87D2355fAf6192"

try:
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "Hello, world!"}]
    )
    print(response)
except Exception as e:
    print(f"API Error: {e}")


{
  "id": "chatcmpl-AX97GB7Ha3Azzvqda3MvaSRT6Z5I2",
  "object": "chat.completion",
  "created": 1732463230,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 11,
    "completion_tokens": 9,
    "total_tokens": 20,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "system_fingerprint": "fp_04751d0b65"
}


In [243]:
#API URL
#https://free.v36.cm
#API KEY
#sk-RFCn5XAIanQqIw5eB0C58d15328246769b87D2355fAf6192

In [244]:
import csv

condition_mapping_file = "../../resources/CCSCM.csv"
procedure_mapping_file = "../../resources/CCSPROC.csv"
drug_file = "../../resources/ATC.csv"

condition_dict = {}
with open(condition_mapping_file, newline='', encoding='utf-8') as csvfile:  # 指定編碼
    reader = csv.DictReader(csvfile)
    for row in reader:
        condition_dict[row['code']] = row['name'].lower()

procedure_dict = {}
with open(procedure_mapping_file, newline='', encoding='utf-8') as csvfile:  # 指定編碼
    reader = csv.DictReader(csvfile)
    for row in reader:
        procedure_dict[row['code']] = row['name'].lower()

drug_dict = {}
with open(drug_file, newline='', encoding='utf-8') as csvfile:  # 指定編碼
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['level'] == '3.0':
            drug_dict[row['code']] = row['name'].lower()

In [245]:
import re 
from ChatGPT import ChatGPT
import json

def extract_data_in_brackets(input_string):
    pattern = r"\[(.*?)\]"
    matches = re.findall(pattern, input_string)
    return matches

def divide_text(long_text, max_len=800):
    sub_texts = []
    start_idx = 0
    while start_idx < len(long_text):
        end_idx = start_idx + max_len
        sub_text = long_text[start_idx:end_idx]
        sub_texts.append(sub_text)
        start_idx = end_idx
    return sub_texts

def filter_triples(triples):
    chatgpt = ChatGPT()
    response = chatgpt.chat(
        f"""
            I have a list of triples. I want to select 50 most important triples from the list.
            The importance of a triple is based on how you think it will help imrpove healthcare prediction tasks (e.g., drug recommendation, mortality prediction, readmission prediction …).
            If you think a triple is important, please keep it. Otherwise, please remove it.
            You can also add triples from your background knowledge.
            The total size of the updated list should be below 50.

            triples: {triples}
            updates:
        """
        )
    json_string = str(response)
    json_data = json.loads(json_string)

    filtered_triples = extract_data_in_brackets(json_data['content'])
    return filtered_triples


In [246]:
from ChatGPT import ChatGPT
import json

def graph_gen(term: str, mode: str):
    if mode == "condition":
        example = """
        Example:
        prompt: systemic lupus erythematosus
        updates: [[systemic lupus erythematosus, is an, autoimmune condition], [systemic lupus erythematosus, may cause, nephritis], [anti-nuclear antigen, is a test for, systemic lupus erythematosus], [systemic lupus erythematosus, is treated with, steroids], [methylprednisolone, is a, steroid]]
        """
    elif mode == "procedure":
        example = """
        Example:
        prompt: endoscopy
        updates: [[endoscopy, is a, medical procedure], [endoscopy, used for, diagnosis], [endoscopic biopsy, is a type of, endoscopy], [endoscopic biopsy, can detect, ulcers]]
        """
    elif mode == "drug":
        example = """
        Example:
        prompt: iobenzamic acid
        updates: [[iobenzamic acid, is a, drug], [iobenzamic acid, may have, side effects], [side effects, can include, nausea], [iobenzamic acid, used as, X-ray contrast agent], [iobenzamic acid, formula, C16H13I3N2O3]]
        """
    chatgpt = ChatGPT()
    response = chatgpt.chat(
        f"""
            Given a prompt (a medical condition/procedure/drug), extrapolate as many relationships as possible of it and provide a list of updates.
            The relationships should be helpful for healthcare prediction (e.g., drug recommendation, mortality prediction, readmission prediction …)
            Each update should be exactly in format of [ENTITY 1, RELATIONSHIP, ENTITY 2]. The relationship is directed, so the order matters.
            Both ENTITY 1 and ENTITY 2 should be noun.
            Any element in [ENTITY 1, RELATIONSHIP, ENTITY 2] should be conclusive, make it as short as possible.
            Do this in both breadth and depth. Expand [ENTITY 1, RELATIONSHIP, ENTITY 2] until the size reaches 100.

            {example}

            prompt: {term}
            updates:
        """
    )
    
    # 印出原始回應檢查格式
    print("Response from ChatGPT:")
    print(response)

    # 去除非關聯的文字描述，使用正則表達式提取關聯三元組
    pattern = r"\[(.*?)\]"  # 匹配 [ENTITY 1, RELATIONSHIP, ENTITY 2] 格式的部分
    matches = re.findall(pattern, response)
    
    # 將符合的部分轉換為三元組格式
    triples = []
    for match in matches:
        entities = match.split(', ')
        if len(entities) == 3:
            triples.append(f"[{entities[0]}, {entities[1]}, {entities[2]}]")
    
    # 輸出過濾後的結果
    print("Filtered triples:")
    for triple in triples:
        print(triple)

    # 返回格式化的資料
    outstr = "\n".join(triples)
    return outstr



In [247]:
## Future work - Including Clinical Notes
# import json

# with open('../../clinical_notes/subject_text_dict.json', 'r') as f:
#     subject_text_dict = json.load(f)

In [248]:
import os

# 定義多個目錄路徑
directories = [
    '../../graphs/condition/CCSCM/',
    '../../graphs/procedure/CCSPROC/',
    '../../graphs/drug/ATC5/',
    '../../graphs/drug/ATC3/'
]

# 檢查每個目錄，若不存在則創建
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")
    else:
        print(f"Directory already exists: {directory}")


Directory already exists: ../../graphs/condition/CCSCM/
Directory already exists: ../../graphs/procedure/CCSPROC/
Directory already exists: ../../graphs/drug/ATC5/
Directory already exists: ../../graphs/drug/ATC3/


In [249]:
from tqdm import tqdm
import os

for key in tqdm(condition_dict.keys()):
    file = f'../../graphs/condition/CCSCM/{key}.txt'
    
    # 檢查檔案是否存在
    if os.path.exists(file):
        with open(file=file, mode="r", encoding='utf-8') as f:
            prev_triples = f.read()
        
        # 檢查檔案行數，若小於 100 行，則添加新內容
        if len(prev_triples.split('\n')) < 100:
            outstr = graph_gen(term=condition_dict[key], mode="condition")
            # 將之前的內容與新內容組合
            outstr = prev_triples + outstr
            # 寫入新內容
            with open(file=file, mode='w', encoding='utf-8') as outfile:
                outfile.write(outstr)
                print(f"Content written to {file}")
    else:
        # 若檔案不存在，直接寫入新內容
        outstr = graph_gen(term=procedure_dict[key], mode="procedure")
        outfile = open(file=file, mode='w', encoding='utf-8')
        outstr = outstr
        # print(outstr)
        outfile.write(outstr)


  0%|          | 1/285 [00:13<1:01:57, 13.09s/it]

Response from ChatGPT:
1. [tuberculosis, is an, infectious disease]  
2. [tuberculosis, is caused by, Mycobacterium tuberculosis]  
3. [tuberculosis, may affect, lungs]  
4. [tuberculosis, can lead to, pneumonia]  
5. [tuberculosis, is spread by, airborne droplets]  
6. [latent tuberculosis, is a type of, tuberculosis]  
7. [tuberculosis, is diagnosed by, skin test]  
8. [tuberculosis, may cause, coughing]  
9. [tuberculosis, may cause, weight loss]  
10. [tuberculosis, may cause, night sweats]  
11. [tuberculosis, can be treated with, antibiotics]  
12. [isoniazid, is an, antibiotic for tuberculosis]  
13. [rifampicin, is an, antibiotic for tuberculosis]  
14. [tuberculosis, can reoccur after, treatment failure]  
15. [drug-resistant tuberculosis, is a form of, tuberculosis]  
16. [tuberculosis, can be diagnosed by, chest X-ray]  
17. [tuberculosis, is more prevalent in, immunocompromised patients]  
18. [tuberculosis, is associated with, HIV infection]  
19. [tuberculosis, can lead t

  1%|          | 2/285 [00:23<54:18, 11.51s/it]  

Response from ChatGPT:
1. [thyroidectomy, is a, surgical procedure]
2. [thyroidectomy, treats, thyroid disorders]
3. [thyroidectomy, involves removal of, thyroid gland]
4. [partial thyroidectomy, is a type of, thyroidectomy]
5. [complete thyroidectomy, is a type of, thyroidectomy]
6. [thyroidectomy, may lead to, hypothyroidism]
7. [hormone replacement therapy, is used for, hypothyroidism]
8. [thyroidectomy, has risks of, complications]
9. [surgical risk, includes, infection]
10. [surgical risk, includes, bleeding]
11. [thyroidectomy, can cause, vocal cord injury]
12. [vocal cord injury, leads to, voice changes]
13. [thyroidectomy, may require, lifelong monitoring]
14. [lifelong monitoring, involves checking for, hormone levels]
15. [thyroidectomy, can improve, hyperthyroid symptoms]
16. [hyperthyroid symptoms, includes, weight loss]
17. [thyroidectomy, may improve, quality of life]
18. [thyroidectomy, is indicated for, thyroid cancer]
19. [thyroid cancer, can spread to, lymph nodes]
20

  1%|          | 3/285 [00:32<48:56, 10.41s/it]

...

Response from ChatGPT:
1. [electrocardiogram, is a, diagnostic test]
2. [electrocardiogram, measures, cardiac activity]
3. [electrocardiogram, detects, arrhythmias]
4. [electrocardiogram, identifies, myocardial infarction]
5. [arrhythmias, can lead to, heart failure]
6. [myocardial infarction, increases risk of, mortality]
7. [electrocardiogram, is indicated for, chest pain]
8. [electrocardiogram, is used in, stress testing]
9. [stress testing, assesses, exercise capacity]
10. [electrocardiogram, can indicate, electrolyte imbalance]
11. [electrocardiogram, monitors, heart rate]
12. [elevated troponin, correlates with, myocardial injury]
13. [myocardial injury, can be diagnosed by, electrocardiogram]
14. [electrocardiogram, can suggest, cardiac ischemia]
15. [cardiac ischemia, requires, intervention]
16. [intervention, can involve, angioplasty]
17. [angioplasty, is a type of, cardiac procedure]
18. [electrocardiogram, can help predict, arrhythmia risk]
19. [arrhythmia risk, is influence

 41%|████      | 117/285 [23:47<32:59, 11.79s/it]

Response from ChatGPT:
1. [electrographic cardiac monitoring, is a, diagnostic tool]
2. [electrographic cardiac monitoring, detects, arrhythmias]
3. [electrographic cardiac monitoring, is used in, hospitals]
4. [electrographic cardiac monitoring, requires, electrodes]
5. [electrographic cardiac monitoring, can predict, heart failure]
6. [electrographic cardiac monitoring, monitors, heart activity]
7. [electrographic cardiac monitoring, provides, real-time data]
8. [electrographic cardiac monitoring, helps assess, patient status]
9. [electrographic cardiac monitoring, informs, treatment decisions]
10. [electrographic cardiac monitoring, can detect, ischemia]
11. [electrographic cardiac monitoring, evaluates, arrhythmia episodes]
12. [electrographic cardiac monitoring, is performed by, healthcare professionals]
13. [electrographic cardiac monitoring, supports, pacemaker functioning]
14. [electrographic cardiac monitoring, analyzes, electrical impulses]
15. [electrographic cardiac monitor

 41%|████      | 117/285 [23:59<34:27, 12.31s/it]


KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm
import os

for key in tqdm(procedure_dict.keys()):
    file = f'../../graphs/procedure/CCSPROC/{key}.txt'
    if os.path.exists(file):
        with open(file=file, mode="r", encoding='utf-8') as f:
            prev_triples = f.read()
        if len(prev_triples.split('\n')) < 150:
            outstr = graph_gen(term=procedure_dict[key], mode="procedure")
            outfile = open(file=file, mode='w', encoding='utf-8')
            outstr = prev_triples + outstr
            # print(outstr)
            outfile.write(outstr)
    else:
        outstr = graph_gen(term=procedure_dict[key], mode="procedure")
        outfile = open(file=file, mode='w', encoding='utf-8')
        outstr = outstr
        # print(outstr)
        outfile.write(outstr)

In [ ]:
from tqdm import tqdm
import os

for key in tqdm(drug_dict.keys()):
    file = f'../../graphs/drug/ATC5/{key}.txt'
    if os.path.exists(file):
        with open(file=file, mode="r", encoding='utf-8') as f:
            prev_triples = f.read()
        if len(prev_triples.split('\n')) < 150:
            outstr = graph_gen(term=drug_dict[key], mode="drug")
            outfile = open(file=file, mode='w', encoding='utf-8')
            outstr = prev_triples + outstr
            # print(outstr)
            outfile.write(outstr)
        # continue
    else:
        outstr = graph_gen(term=drug_dict[key], mode="drug")
        outfile = open(file=file, mode='w', encoding='utf-8')
        outstr = outstr
        # print(outstr)
        outfile.write(outstr)

In [ ]:
from tqdm import tqdm
import os

for key in tqdm(drug_dict.keys()):
    file = f'../../graphs/drug/ATC3/{key}.txt'
    if os.path.exists(file):
        with open(file=file, mode="r", encoding='utf-8') as f:
            prev_triples = f.read()
        if len(prev_triples.split('\n')) < 150:
            outstr = graph_gen(term=drug_dict[key], mode="drug")
            outfile = open(file=file, mode='w', encoding='utf-8')
            outstr = prev_triples + outstr
            # print(outstr)
            outfile.write(outstr)
        # continue
    else:
        outstr = graph_gen(term=drug_dict[key], mode="drug")
        outfile = open(file=file, mode='w', encoding='utf-8')
        outstr = outstr
        # print(outstr)
        outfile.write(outstr)